In [12]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression, LinearRegression
from sklearn.neural_network import MLPClassifier, MLPRegressor
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score, mean_squared_error
from xgboost import XGBRegressor

# Đọc dữ liệu từ tệp CSV
try:
    data = pd.read_csv('../../../data/BCP.csv')
    output_file = '../../../data/BCP_predictions.csv'
    # Xem trước dữ liệu
    print("Dữ liệu gốc:")
    print(data.head())
    print(data.columns)

    # Loại bỏ cột 'Time' vì không sử dụng trong tính toán
    data = data.drop(columns=['Time'])

    # Kiểm tra giá trị NaN
    print("\nKiểm tra giá trị NaN:")
    print(data.isna().sum())

    # Xóa các hàng chứa giá trị NaN
    data.dropna(inplace=True)

    # Giả sử chúng ta sẽ chọn cột '440FI539' làm mục tiêu và các cột còn lại là đặc trưng
    target_column = '440FI539'  # Bạn có thể chọn cột khác nếu cần
    features = [col for col in data.columns if col != target_column]

    # Chuyển đổi cột mục tiêu thành nhị phân (0 hoặc 1) dựa trên giá trị trung bình
    data[f'{target_column}_binary'] = (data[target_column] > data[target_column].mean()).astype(int)

    X = data[features]
    y = data[f'{target_column}_binary']

    # Chia dữ liệu thành tập huấn luyện và tập kiểm tra
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

    # Chuẩn hóa dữ liệu
    scaler = StandardScaler()
    X_train = scaler.fit_transform(X_train)
    X_test = scaler.transform(X_test)

    # Hồi quy đơn biến (Univariate Regression)
    univariate_model = LinearRegression()
    univariate_model.fit(X_train[:, 0].reshape(-1, 1), y_train)
    y_pred_univariate = univariate_model.predict(X_test[:, 0].reshape(-1, 1))
    mse_univariate = mean_squared_error(y_test, y_pred_univariate) * 100
    print("Univariate Regression")
    print(f"MSE: {mse_univariate:.6f}%")

    # Hồi quy đa biến (Multivariate Regression)
    multivariate_model = LinearRegression()
    multivariate_model.fit(X_train, y_train)
    y_pred_multivariate = multivariate_model.predict(X_test)
    mse_multivariate = mean_squared_error(y_test, y_pred_multivariate) * 100
    print("Multivariate Regression")
    print(f"MSE: {mse_multivariate:.6f}%")

    # Huấn luyện mô hình hồi quy Logistic
    logistic_model = LogisticRegression()
    logistic_model.fit(X_train, y_train)

    # Dự đoán với mô hình hồi quy Logistic
    y_pred_logistic = logistic_model.predict(X_test)

    # Huấn luyện mô hình MLP
    mlp_model = MLPClassifier(hidden_layer_sizes=(10,), max_iter=1000, random_state=42)
    mlp_model.fit(X_train, y_train)

    # Dự đoán với mô hình MLP
    y_pred_mlp = mlp_model.predict(X_test)

    # Kết hợp kết quả dự đoán từ cả hai mô hình
    y_pred_combined = (y_pred_logistic + y_pred_mlp) / 2
    y_pred_combined = np.round(y_pred_combined).astype(int)

    # Đánh giá độ chính xác, F1 score, Precision và Recall
    accuracy = accuracy_score(y_test, y_pred_combined) * 100
    f1 = f1_score(y_test, y_pred_combined) * 100
    precision = precision_score(y_test, y_pred_combined) * 100
    recall = recall_score(y_test, y_pred_combined) * 100

    # XGBoost
    xgboost_model = XGBRegressor()
    xgboost_model.fit(X_train, y_train)
    y_pred_xgboost = xgboost_model.predict(X_test)
    mse_xgboost = mean_squared_error(y_test, y_pred_xgboost) * 100
    print("XGBoost")
    print(f"MSE: {mse_xgboost:.6f}%")

    # MLP (Multi-Layer Perceptron)
    mlp_reg_model = MLPRegressor(hidden_layer_sizes=(10,), max_iter=2000, random_state=42)
    mlp_reg_model.fit(X_train, y_train)
    y_pred_mlp_reg = mlp_reg_model.predict(X_test)
    mse_mlp_reg = mean_squared_error(y_test, y_pred_mlp_reg) * 100
    print("MLP")
    print(f"MSE: {mse_mlp_reg:.6f}%")

    # Mô hình kết hợp (Hybrid Model) - Kết hợp Multivariate Regression và MLP
    y_pred_combined_reg = (y_pred_multivariate + y_pred_xgboost + y_pred_mlp_reg) / 3
    mse_combined_reg = mean_squared_error(y_test, y_pred_combined_reg) * 100
    print("Hybrid Model")
    print(f"MSE: {mse_combined_reg:.6f}%")

    # Thêm cột dự đoán vào dữ liệu ban đầu với tên cột mới
    X_test_df = pd.DataFrame(X_test, columns=features)
    data.loc[X_test_df.index, f'{target_column}_prediction'] = y_pred_combined

    data.to_csv(output_file, index=False)

    # Hiển thị kết quả
    print(f"Accuracy: {accuracy:.2f}%")
    print(f"F1 Score: {f1:.2f}%")
    print(f"Precision: {precision:.2f}%")
    print(f"Recall: {recall:.2f}%")
    print(f"Data Result with Predictions:\n{data.head()}")

    # Tổng kết kết quả
    print("\nSummary of Results")
    print("Univariate Regression")
    print(f"MSE: {mse_univariate:.6f}%")

    print("\nMultivariate Regression")
    print(f"MSE: {mse_multivariate:.6f}%")

    print("\nXGBoost")
    print(f"MSE: {mse_xgboost:.6f}%")

    print("\nMLP")
    print(f"MSE: {mse_mlp_reg:.6f}%")

    print("\nHybrid Model")
    print(f"MSE: {mse_combined_reg:.6f}%")

except Exception as e:
    print(f"Error processing the data: {e}")


ModuleNotFoundError: No module named 'xgboost'